# Transformer Language Models

Somewhat basic implemention of transformer model

In [ ]:
#| default_exp models.transformer

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export

# torch
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.optim import SGD

# hf
import datasets

# data 
import pandas as pd
import numpy as np

# ui
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

# python
from typing import Dict, List, Tuple, Optional, Set
from collections import Counter, OrderedDict
from dataclasses import dataclass, asdict
from plum import dispatch
from pathlib import Path

# nimrod
# from nimrod.models.lm import Vocab
from nimrod.text.tokenizers import CharTokenizer
from nimrod.text.datasets import SimpleCharDataset
from nimrod.utils import get_device

In [ ]:
device = get_device()
print(device)

[11:12:56] INFO - Using device: mps


mps


## Data formatting
- https://buomsoo-kim.github.io/attention/2020/04/21/Attention-mechanism-19.md/


In [ ]:
# dataset = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1')

In [ ]:
data = Path('../data/text/tiny_shakespeare.txt').read_text()
tokenizer = CharTokenizer.from_text(data)
tokenized = tokenizer.encode(data)
ds = SimpleCharDataset(tokenized, context_length=8)
x,y = ds[0]
print(tokenizer.decode(x), tokenizer.decode(y))

First Ci irst Cit


## Attention

Imagine you're at position "it" in:

"The cat sat on the mat because it was tired."

- Q vector for "it" says "I need an antecedent".
- It matches strongest with K vector from "cat".
- So V vector from "cat" is heavily weighted in the output for "it".

In [ ]:
#| export

class AttentionHead(nn.Module):
    """ self attention head """
    def __init__(self, n_embd, head_size, block_size, dropout):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels/n_embd)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [ ]:
vocab_size = 10
batch_size = 5
embed_dim = 20
context_size = 8
dropout = 0.2
head_size = 16
# embedded input (float)
x = torch.randn(batch_size, context_size, embed_dim) #(B,T,C)
print(x.shape)
att = AttentionHead(embed_dim, head_size, context_size, dropout)
xx = att(x)
print(xx.shape) # (B, T, Head_size)

torch.Size([5, 8, 20])
torch.Size([5, 8, 16])


In [ ]:
#| export

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size, embed_dim, block_size, dropout):
        super().__init__()
        self.heads = nn.ModuleList([AttentionHead(embed_dim, head_size, block_size, dropout) for _ in range(num_heads)]) # each head: B,T,Head_size
        self.proj = nn.Linear(head_size * num_heads, embed_dim) # actually head_size*num_heads == embed_dim
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        heads_output = [h(x) for h in self.heads]
        out = torch.cat(heads_output, dim=-1) # concatenate along last dim (head_size) to get head_size*num_heads=embed_dim
        out = self.dropout(self.proj(out))
        return out

In [ ]:
num_heads = 5
multi_att = MultiHeadAttention(num_heads, head_size, embed_dim, context_size, dropout)
xxx = multi_att(x)
print(xxx.shape) #B,T,C/embed_dim

torch.Size([5, 8, 20])


In [ ]:
mha = nn.MultiheadAttention(
    embed_dim=embed_dim,
    num_heads=num_heads,
    dropout=dropout,
    batch_first=True
    )
attn_out, attn_weight = mha(x, x, x)
print(attn_out.shape)

torch.Size([5, 8, 20])


## Feed forward

In [ ]:
#| export 

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self,embed_dim, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embed_dim, 4 *embed_dim), # 4*embed_dim somewhat arbitrary. heuristic.
            nn.ReLU(), #nn.GELU
            nn.Linear(4 *embed_dim,embed_dim),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
ff = FeedFoward(embed_dim, dropout)
ff_x = ff(x)
print(ff_x.shape)

torch.Size([5, 8, 20])


## Block

In [ ]:
#| export 

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, embed_dim, n_head, block_size, dropout):
        # embed_dim: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = embed_dim // n_head

        self.ln1 = nn.LayerNorm(embed_dim) 
        self.sa = MultiHeadAttention(n_head, head_size, embed_dim, block_size, dropout) # B, T, C/embed_dim
        self.ln2 = nn.LayerNorm(embed_dim)
        self.ffwd = FeedFoward(embed_dim, dropout) # B, T, C

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
b = Block(embed_dim, num_heads, context_size, dropout)
bb = b(x)
print(bb.shape)

torch.Size([5, 8, 20])


## GPT model

In [ ]:
#| export

class GPTLanguageModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, block_size, n_head, n_layer, dropout):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.block_size = block_size
        self.token_embedding_table = nn.Embedding(vocab_size, embed_dim)
        self.position_embedding_table = nn.Embedding(block_size, embed_dim)
        self.blocks = nn.Sequential(*[Block(embed_dim, n_head, block_size, dropout) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(embed_dim) # final layer norm
        self.lm_head = nn.Linear(embed_dim, vocab_size) # back to vocab size dim

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, x, targets=None):
        B, T = x.shape #x is token IDs

        # x and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(x) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=x.device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, ids, max_new_tokens):
        # ids is (B, T) arr #ids is token ID say of indices in the current context
        for _ in range(max_new_tokens):
            # crop ids to the last block_size tokens
            ids_cond = ids[:, -self.block_size:]
            # get the predictions
            logits, loss = self(ids_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            ids_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            ids = torch.cat((ids, ids_next), dim=1) # (B, T+1)
        return ids #ids is token IDs

In [ ]:
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
eval_interval = 500
learning_rate = 3e-4
device = get_device()
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
vocab_size = 46 #len(v)
m = GPTLanguageModel(vocab_size, n_embd, block_size, n_head, n_layer, dropout)

[11:25:24] INFO - Using device: mps


In [ ]:
print(device)

mps


In [ ]:
device = 'cpu'
m = m.to(device)
x = torch.randint(vocab_size, (batch_size, block_size)).to(device)
logits, loss = m(x)
print(logits.shape)

torch.Size([64, 256, 46])


In [ ]:
# @torch.no_grad()
# def estimate_loss():
#     out = {}
#     model.eval()
#     for split in ['train', 'val']:
#         losses = torch.zeros(eval_iters)
#         for k in range(eval_iters):
#             X, Y = get_batch(split)
#             logits, loss = model(X, Y)
#             losses[k] = loss.item()
#         out[split] = losses.mean()
#     model.train()
#     return out

In [ ]:
#| notest
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
acc_loss = []
max_iters = 1
for iter in range(max_iters):
    # sample a batch of data
    xb, yb = get_random_batch(torch.LongTensor(ids), block_size, batch_size, device=device)
    # evaluate the loss
    logits, loss = m(xb.to(device), yb.to(device))
    print(loss.item())
    acc_loss.append(loss.item())
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
#| notest
plt.plot(acc_loss)

In [ ]:
#| notest
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(''.join(v.itos(m.generate(context, max_new_tokens=50)[0].tolist())))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()